# <span style="color:blue"> **Assignment 2: Multivariate Linear Regression**</span>

## <span style="color:blue"> **Modules & Packages**</span>

In [1]:
# Pandas: To manipulate DataFrames
import pandas as pd
# Seaborn & Matplotlib: To plot scatter plots
import seaborn as sns
import matplotlib.pyplot as plt
# Statsmodels: To estimate our regression model
import statsmodels.api as sm
# Numpy: For mathematical computations
import numpy as np

## <span style="color:blue"> **The Dataset**</span>

In [3]:
sheet = ["Y variable", "x1-market", "x2-value", "x3-small"]
sets = []

data_dict = {}
for title in sheet:
    data_dict[title] = pd.DataFrame(pd.read_excel("TP2-Dataset.xlsx", skiprows = range(6), sheet_name=title).iloc[:]['PX_LAST'])
    data_dict[title][title] = np.log(data_dict[title]['PX_LAST']/data_dict[title]['PX_LAST'].shift(1))
    del data_dict[title]['PX_LAST']
    data_dict[title] = data_dict[title].dropna()
    sets += [data_dict[title]]

df = pd.concat(sets, axis = 1)

In [4]:
name = ['Tech', 'Market', 'Value', 'Size']

ren = {sheet[i]: name for i, name in enumerate(name)}
df = df.rename(columns=ren)

In [7]:
with open('latex/descriptive_stats.txt', 'w') as txt_file:
    txt_file.write(df.describe().to_latex())

df.describe()

,Tech,Market,Value,Size
count,517.000000,517.000000,517.000000,517.000000
mean,-0.003133,-0.000783,-0.000799,-0.000167
std,0.031860,0.019804,0.022181,0.027590
min,-0.151192,-0.090930,-0.095603,-0.134211
25%,-0.021413,-0.011619,-0.012017,-0.014185
50%,-0.004975,-0.002268,-0.001781,-0.002370
75%,0.013590,0.007087,0.007886,0.012890
max,0.140315,0.165685,0.211631,0.264327


## <span style="color:blue"> **Multivariate OLS Regression**</span>

In [8]:
# Define dependant (y) and independent (x) variables
y = df[name[0]]
x = df[name[1:]]

# Add a constant to the predictor variable (intercept)
x = sm.add_constant(x)

# Fit the OLS model to the sample data
model_ols = sm.OLS(y, x).fit()

with open('latex/model_summary.txt', 'w') as txt_file:
    for table in model_ols.summary().tables:
        txt_file.write(table.as_latex_tabular())

# View model summary (intercept, coefficient, r-squared)
model_ols.summary()

C:\Users\joelc\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Tech   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.496
Method:                 Least Squares   F-statistic:                     170.1
Date:                Sat, 08 Oct 2022   Prob (F-statistic):           1.56e-76
Time:                        18:01:09   Log-Likelihood:                 1227.2
No. Observations:                 517   AIC:                            -2446.
Df Residuals:                     513   BIC:                            -2429.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0022      0.001     -2.247      0.025      -0.004      -0.000
Market         2.8998      0.166     17.423      0.000       2.573       3.227
Value         -1.6893      0.131    -12.932      0.000      -1.946      -1.433
Size          -0.1698      0.072     -2.352      0.019      -0.312      -0.028
==============================================================================
Omnibus:                       12.445   Durbin-Watson:                   2.194
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               23.633
Skew:                          -0.007   Prob(JB):                     7.38e-06
Kurtosis:                       4.047   Cond. No.                         204.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## <span style="color:blue"> **OLS Properties**</span>

In [10]:
# Property 1: The sum of residuals must be null
print(np.sum(model_ols.resid))

-1.1102230246251565e-15


> This essentially means the sum of residuals is 0. The first property is therefore verified.

In [12]:
# Property 2: The covariances between the residuals and each explanatory variable must all be null

col_names = ["Market", "Value", "Size"]

for col in col_names:
    cov = df[col].cov(model_ols.resid)
    print(f"Covariance between {col} and residuals: {cov}")

Covariance between Market and residuals: 6.7478978235217e-19
Covariance between Value and residuals: 8.33532949273255e-19
Covariance between Size and residuals: 8.640523999619216e-19


> The sample covariances between each explanatory variable and the model's residuals are all null. The second OLS property is thus verified.

In [16]:
# Property 3: The regression line must pass through the average values of the depedent variable and each explanatory variable
# Store mean of each variable
tech_mean = df.Tech.mean()
market_mean = df.Market.mean()
value_mean = df.Value.mean()
size_mean = df.Size.mean()

# Store estimators
params = model_ols.params

# Verify the property
print(f"Sample average for Tech: {tech_mean}") 
print(f"Regression line passing through sample averages of the explan. variables: {params[0] + params[1]*market_mean + params[2]*value_mean + params[3]*size_mean}")

Sample average for Tech: -0.0031332184926364305
Regression line passing through sample averages of the explan. variables: -0.0031332184926364283


> These two values are approxiamtely equal. The third OLS property therefore holds.